# Wing Model Analysis

This Jupyter Notebook provides a analysis of the trained wing model. The notebook is structured to load, preprocess, and analyze wing data, and it includes the following key components:

1. **Imports and Setup**:
    - Import necessary libraries and modules.
    - Define paths for project, data, models, and results.
    - Set up configurations for plotting and saving results.

2. **Date and Device Setup**:
    - Get the current date.
    - Check if a GPU is available and set the device accordingly.

3. **Utility Functions and Model Imports**:
    - Append the project path to the system path.
    - Import utility functions and models for airfoil and wing analysis.

4. **Airfoil Model Initialization**:
    - Initialize and load pre-trained airfoil models for lift and drag coefficients.
    - Define classes for airfoil models using RBF networks.

5. **Wing Dataset Preparation**:
    - Define a custom `WingDataset` class to load and preprocess wing data from CSV files.
    - Extract relevant features and target variables, including aerodynamic coefficients and geometry parameters.

6. **LSTM Model with Attention**:
    - Define an LSTM model with multi-head attention for predicting aerodynamic coefficients.
    - Initialize and load a pre-trained wing model.

7. **Evaluation and Metrics Calculation**:
    - Define functions to compute evaluation metrics such as MAPE and relative L2 norm error.
    - Load test datasets and evaluate the model's performance on these datasets.
    - Plot and save the results, including comparisons between predicted and actual aerodynamic coefficients.

8. **Results and Summary**:
    - Compute and display overall metrics across all test cases.
    - Save the evaluation results to a CSV file if configured to do so.



In [ ]:
import os
import pandas as pd
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from scipy.fftpack import fft, ifft
from scipy.interpolate import CubicSpline

import matplotlib.pyplot as plt
import sys

import joblib

In [ ]:
project_path = "/mnt/e/eVTOL_model/eVTOL-VehicleModel/"     # path to the project
save_path = project_path + "result/wing_model/"            # path to save the result   
model_path = project_path + "trained_models/"               # path to the saved model and scaler
data_path = project_path + "data/wing_data/"                


PLOT_RESULT = True      # plot the result
SAVE_PLOT = False       # save the plot
SAVE_RESULT = False     # save the result

In [ ]:
import datetime
date = datetime.date.today()

In [ ]:
# Check if GPU is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
sys.path.append(project_path + '/src')


# Import necessary functions
from utility_functions import downsample_to_35
from utility_functions import organize_data

# Import all the models
from af_escnn_cl import ESCNN_Cl
from af_escnn_cd import ESCNN_Cd

from af_rbf_cl import RBFLayer_cl, RBFNet_cl
from af_rbf_cd import RBFLayer_cd, RBFNet_cd

In [ ]:
# Initialize airfoil models
root_airfoilModelsTrained = model_path + '/models/airfoil/'


# Load the model weights
af_model_ESCNN_Cl = ESCNN_Cl()
af_model_ESCNN_Cl.load_state_dict(torch.load(root_airfoilModelsTrained + '2024-11-18_model_Cl_ESCNN_lr1e-05_e1500_rbf170_convL4.pth'))
af_model_ESCNN_Cl = af_model_ESCNN_Cl.to(device)
af_model_ESCNN_Cl.eval()

# Load the model weights
af_model_ESCNN_Cd = ESCNN_Cd()
af_model_ESCNN_Cd.load_state_dict(torch.load(root_airfoilModelsTrained + '2024-11-18_model_Cd_ESCNN_lr5e-05_e250_convL3.pth'))
af_model_ESCNN_Cd = af_model_ESCNN_Cd.to(device)
af_model_ESCNN_Cd.eval()

input_size = 140
output_size = 4
num_rbf_units = 4

kmeans_center_cl = torch.load(root_airfoilModelsTrained + '2024-11-19_airfoil_model_Cl_ESCNN_RBF_lr0.06_epoch200_rbfUnits4_RBFcenters.pth')
kmeans_center_cd = torch.load(root_airfoilModelsTrained + '2024-11-19_airfoil_model_Cd_ESCNN_RBF_lr0.025_epoch100_rbfUnits4_RBFcenters.pth')


class airfoilModel_cl(RBFNet_cl):
    def __init__(self):
        super(airfoilModel_cl, self).__init__(input_size, num_rbf_units, output_size, kmeans_center_cl)


class airfoilModel_cd(RBFNet_cd):
    def __init__(self):
        super(airfoilModel_cd, self).__init__(input_size, num_rbf_units, output_size, kmeans_center_cd)


# Initialize the model
airfoil_cl = airfoilModel_cl()
airfoil_cl.load_state_dict(torch.load(root_airfoilModelsTrained + '2024-11-19_airfoil_model_Cl_ESCNN_RBF_lr0.06_epoch200_rbfUnits4.pth'))

airfoil_cd = airfoilModel_cd()
airfoil_cd.load_state_dict(torch.load(root_airfoilModelsTrained + '2024-11-19_airfoil_model_Cd_ESCNN_RBF_lr0.025_epoch100_rbfUnits4.pth'))

airfoil_cl = airfoil_cl.to(device)
airfoil_cd = airfoil_cd.to(device)

airfoil_cl.eval()
airfoil_cd.eval()

In [ ]:
# Prepare wing dataset

# Condition function to filter subdirectories
def subdir_condition(subdir_name):
    """
    Condition: Only process subdirectories whose names start with 'wing_dataset'.
    Modify this function to apply a specific filtering logic.
    """
    return subdir_name.startswith('wing_dataset')  # Change this condition as needed


class WingDataset(Dataset):
    def __init__(self, root_dir, af_data_path= project_path + '/data/wing_section/small_database_testing_csv', subdir_condition=None):
        """
        Args:
            root_dir (string): Root directory with subdirectories containing CSV files.
            af_data_path (string): Airfoil polar database directory contaiing airfoil polars of used airfoils.            
            subdir_condition (callable, optional): A function or condition to filter subdirectories by name.
        """
        self.root_dir = root_dir
        self.data = []
        self.targets = []
        self.time_data = []  # Store time data separately
        self.AOA_data = []
        self.v_inf_data = []
        self.cl_data = []
        self.cd_data = []

        self.cl_data = []
        self.cd_data = []
        self.fft_cl_r = []
        self.fft_cl_i = []
        self.fft_cd_r = []
        self.fft_cd_i = []
        
        self.subdir_condition = subdir_condition
        self.af_data_path = af_data_path

        # Traverse the root directory to gather data
        self._load_data()

    def _load_data(self):
        """
        Helper function to read CSV files from each subdirectory and extract relevant columns.
        """
        # Iterate through each subdirectory in the root directory
        for subdir, _, files in os.walk(self.root_dir):
            subdir_name = os.path.basename(subdir)
            
            # Apply subdirectory name condition
            if self.subdir_condition and not self.subdir_condition(subdir_name):
                continue

            for file in files:

                if file.endswith("_convergence.csv"):
                    # Load the CSV file
                    csv_path = os.path.join(subdir, file)
                    df = pd.read_csv(csv_path)
                    
                    # Extract necessary columns for input features from wing_convergence.csv
                    time = df['T'].values  # Time
                    
                    v_inf = df['Vinf'].values
                    AOA = df['alpha(eff)'].values
                    theta = df['theta'].values
                    vz = df['vVehicle_z'].values
                                        
                    # Extract Cl and Cq for output variables
                    cl = df['CL'].values  # Lift coefficient (CL)
                    cd = df['CD'].values  # Drag coefficient (CD)

                    fft_cl = fft(cl)
                    fft_cl_real = np.real(fft_cl)
                    fft_cl_imag = np.imag(fft_cl)

                    fft_cd = fft(cd)
                    fft_cd_real = np.real(fft_cd)
                    fft_cd_imag = np.imag(fft_cd)

                    # Add the geometry parameters
                    geom_data_path = os.path.join(subdir, 'wing_geometry_data.csv')         # Read from wing_geometry_data.csv file - b, ar, tr, sweep, dihedral
                    geom_df = pd.read_csv(geom_data_path, nrows=1)

                    ones_empty = np.ones_like(time)                                         # Empty array of ones
                    
                    b_data = geom_df["b_wing"].values
                    b_data_array = geom_df["b_wing"].values * ones_empty                    # Making this an array simplifies things later
                    
                    ar_data = geom_df["ar_wing"].values
                    ar_data_array = geom_df["ar_wing"].values * ones_empty
                    
                    tr_data = geom_df["tr"].values
                    tr_data_array = geom_df["tr"].values * ones_empty

                    sweep_data = geom_df["lambda"].values
                    sweep_data_array = geom_df["lambda"].values * ones_empty
                    
                    gamma_data = geom_df["gamma"].values
                    gamma_data_array = geom_df["gamma"].values * ones_empty
                    
                    # Store the following in separate lists for easy access
                    self.time_data.append(time)
                    self.AOA_data.append(AOA)
                    self.v_inf_data.append(v_inf)
                    
                    self.cl_data.append(cl)
                    self.cd_data.append(cd)

                    self.fft_cl_r.append(fft_cl_real)
                    self.fft_cl_i.append(fft_cl_imag)
                    self.fft_cd_r.append(fft_cd_real)
                    self.fft_cd_i.append(fft_cd_imag)

                    # Extract the airfoil details
                    af_name = geom_df["airfoil "].values
                    extension = '.csv'
                    af_name = str(af_name[0]+extension)

                    af_name_new = af_name.split('-')
                    af_coordinate = str(af_name_new[1]+'_coordinates.dat')          # Name of airfoil coordinates. Adjust it according to the names being used
                    # print(af_coordinate)
                    
                    af_polar_data = pd.read_csv(os.path.join(self.af_data_path, af_name), skiprows=10)
                    # af_polar_data = af_polar_data[[(af_polar_data["Alpha"] >= -2) & (af_polar_data["Alpha"] <= 12)]]
                    # af_coordinate_data = pd.read_csv(os.path.join(self.af_data_path, af_coordinate), delim_whitespace=True)  # or use delimiter=','
                    af_coordinate_data = np.loadtxt(os.path.join(self.af_data_path, af_coordinate))

                    af_coordinate_x = af_coordinate_data[:,0]
                    af_coordinate_y = af_coordinate_data[:,1] 

                    AOA_af_polar = af_polar_data["Alpha"].values
                    cl_af_polar = af_polar_data["Cl"].values
                    cd_af_polar = af_polar_data["Cd"].values

                    # Fit a polynomial to Cl and Cd data
                    # Create cubic spline interpolation
                    spline_cl_airfoil = CubicSpline(AOA_af_polar, cl_af_polar)
                    spline_cd_airfoil = CubicSpline(AOA_af_polar, cd_af_polar)

                    cl_poly_coeff = np.polyfit(AOA_af_polar, cl_af_polar, deg=6)
                    cd_poly_coeff = np.polyfit(AOA_af_polar, cd_af_polar, deg=6)

                    cl_airfoil_calc = spline_cl_airfoil(AOA)
                    cd_airfoil_calc = spline_cd_airfoil(AOA)

                    #-------------------------------------------------------------------------------------------------------------------
                    # Note - To determine the airfoil aerodynamic coefficients, there are two approaaches.
                    # 1. Use the polar file from the database to fit a polynomial function and then estimate the Cl/Cd for any given AOA.
                    # 2. USe the pre-trained Neural Network to predict the Cl/Cd.  
                    #-------------------------------------------------------------------------------------------------------------------
                    
                    # Using approach 2 - Using RBF NN to predict the Cl and Cd
                    #-------------------------------------------------------------------------------------------------------------------
                
                    degree = 3
                    
                    input_sequence_cl_NN = []
                    input_sequence_cd_NN = []

                    # Prepare the data to input to ESCNN Model
                    x_escnn = np.array(downsample_to_35(af_coordinate_x)).reshape(1, -1)
                    y_escnn = np.array(downsample_to_35(af_coordinate_y)).reshape(1, -1)
                    aoa_escnn = np.array(AOA_af_polar).reshape(1, -1)

                    elements_ESCNN = organize_data(x_escnn, y_escnn, aoa_escnn)

                    if elements_ESCNN.shape != (0,):
                        input_escnn = elements_ESCNN

                        input_escnn = torch.tensor(input_escnn, dtype=torch.float32).to(device)

                        # Evaluate the model on test dataset
                        with torch.no_grad():
                            Cl_escnn_pred = af_model_ESCNN_Cl.forward(input_escnn)
                            Cd_escnn_pred = af_model_ESCNN_Cd.forward(input_escnn)
                            
                        Cl_escnn_pred = Cl_escnn_pred.cpu().detach().numpy()  # Convert tensor to numpy array
                        Cl_escnn_pred = Cl_escnn_pred.squeeze(1)

                        Cd_escnn_pred = Cd_escnn_pred.cpu().detach().numpy()  # Convert tensor to numpy array
                        Cd_escnn_pred = Cd_escnn_pred.squeeze(1)

                        plt_af_polar_comparison = False     # If needed for debugging
                        if plt_af_polar_comparison == True:
                            plt.figure()
                            # plt.plot(aoa_test[j], Cl_escnn_pred)
                            # # plt.plot(alphas_t[0], Cl_eval_org_scale)
                            # plt.plot(aoa_test[j], cl_test[j])

                            plt.legend(['NN Model - ESCNN', 'UIUC database'])
                            # plt.title(r'$C_d$ vs $\alpha$ for {} airfoil'.format(keyword))
                            plt.xlabel(r'AOA [$\alpha$]')
                            plt.ylabel(r'$C_l$')

                    else:
                        continue

                    af_coordinate_x_i = downsample_to_35(af_coordinate_x)
                    af_coordinate_y_i = downsample_to_35(af_coordinate_y)
                    AOA_af_polar_i = downsample_to_35(AOA_af_polar)
                    cl_af_polar_i = downsample_to_35(Cl_escnn_pred)
                    cd_af_polar_i = downsample_to_35(Cd_escnn_pred)

                    input_sequence_cl_NN = [
                                            af_coordinate_x_i, af_coordinate_y_i, AOA_af_polar_i, cl_af_polar_i
                                        ]
                    
                    input_sequence_cd_NN = [
                                            af_coordinate_x_i, af_coordinate_y_i, AOA_af_polar_i, cd_af_polar_i
                                        ]



                    input_sequence_cl_NN = np.array(input_sequence_cl_NN, dtype=float).reshape(1, -1)
                    input_sequence_cd_NN = np.array(input_sequence_cd_NN, dtype=float).reshape(1, -1)
                    
                    # print("Airfoil Cl NN - Input shape: ", input_sequence_cl_NN.shape)
                    # print("Airfoil Cd NN - Input shape: ", input_sequence_cd_NN.shape)

                    NN_cl_model_ip_data = torch.tensor(input_sequence_cl_NN, dtype=torch.float32).to(device) 
                    NN_cd_model_ip_data = torch.tensor(input_sequence_cd_NN, dtype=torch.float32).to(device) 

            
                    # Neural network model to predict the airfoil aerodynamic coeficients    
                    with torch.no_grad():  # Disable gradient computation for inference
                        predicted_coefficients_cl = airfoil_cl(NN_cl_model_ip_data)
                        predicted_coefficients_cd = airfoil_cd(NN_cd_model_ip_data)

                    predicted_af_coefficients_cl = predicted_coefficients_cl.cpu().detach().numpy()
                    predicted_af_coefficients_cd = predicted_coefficients_cd.cpu().detach().numpy()
                    
                    # print(predicted_af_coefficients_cl)

                    # polynomial_cl_new = np.poly1d(predicted_coefficients[0])
                    polynomial_cl_pred = np.poly1d(predicted_af_coefficients_cl[0])
                    polynomial_cd_pred = np.poly1d(predicted_af_coefficients_cd[0])

                    x_new = np.linspace(AOA_af_polar[0], AOA_af_polar[-1], 100)
                    # y_new_cl = polynomial_cl_new(x_new_cl)
                    y_new_cl = polynomial_cl_pred(x_new)
                    y_new_cd = polynomial_cd_pred(x_new)

                    

                    # # plt.figure()
                    plt_af_polar_NN = False
                    if plt_af_polar_NN == True:
                        plt.figure(figsize=(15, 5))
                        # plt.title(af_name)
                        
                        plt.subplot(1,2,1)
                        plt.plot(AOA_af_polar, cl_af_polar, color='red', label='UIUC Database')
                        plt.plot(x_new, y_new_cl, label=f'Polynomial Degree {degree} - NN')
                        plt.xlabel(r'$\alpha$')
                        plt.ylabel(r'$C_l$')
                        plt.title(af_name)
                        plt.legend()


                        plt.subplot(1,2,2)
                        plt.plot(AOA_af_polar, cd_af_polar, color='red', label='UIUC Database')
                        plt.plot(x_new, y_new_cd, label=f'Polynomial Degree {degree} - NN')
                        plt.xlabel(r'$\alpha$')
                        plt.ylabel(r'$C_d$')
                        plt.title(af_name)
                        plt.legend()
                    
                    
                    cl_af_NN = polynomial_cl_pred(AOA)
                    cd_af_NN = polynomial_cd_pred(AOA)


                    # print(cl_af_NN)
                    # print(cl_airfoil_calc)
                    plt_af_lvl_coeff = False
                    if plt_af_lvl_coeff==True:
                        plt.figure(figsize=(15, 5))
                        
                        
                        plt.subplot(1,2,1)
                        plt.plot(time, cl_airfoil_calc, color='red', label='UIUC Database')
                        plt.plot(time, cl_af_NN, label=f'Polynomial Degree {degree} - NN')
                        plt.xlabel(r'$\alpha$')
                        plt.ylabel(r'$C_l$')
                        plt.title(af_name)
                        plt.legend()

                        plt.subplot(1,2,2)
                        plt.plot(time, cd_airfoil_calc, color='red', label='UIUC Database')
                        plt.plot(time, cd_af_NN, label=f'Polynomial Degree {degree} - NN')
                        plt.xlabel(r'$\alpha$')
                        plt.ylabel(r'$C_d$')
                        plt.title(af_name)
                        plt.legend()
                    
                    #-------------------------------------------------------------------------------------------------------------------
                    # Calculate the Cl and Cd using the emperical equations using airfoil aerodynamic coefficients
                    # Calculate Wing Aerodynamic Coefficients

                    vel_comp_factor_cl = 0.01                # Assumed from trial & error. Depends on flow velocity and wing geometry
                    vel_comp_factor_cd = 0.00005              # Assumed from trial & error. Depends on flow velocity and wing geometry
                    e = 0.85                                # Oswald factor

                    # cl_wing_calc = cl_airfoil_calc / (1 + cl_airfoil_calc / (np.pi * ar_data * e)) * vel_comp_factor_cl

                    # cd_induced = (cl_wing_calc ** 2) / (np.pi * ar_data * e) * vel_comp_factor_cd

                    cl_wing_calc = cl_af_NN / (1 + cl_af_NN / (np.pi * ar_data * e)) * vel_comp_factor_cl

                    cd_induced = (cl_wing_calc ** 2) / (np.pi * ar_data * e) * vel_comp_factor_cd

                    # Step 5: Total drag coefficient for the wing
                    cd_wing_calc = cd_af_NN + cd_induced
                    # cd_wing_calc = cd_airfoil_calc + cd_induced

                    # For each simulation, the input sequence is structured as (n_timesteps, n_features)
                    sequence_inputs = []
                    sequence_outputs = []
                    for i in range(len(time)):
                        # Each time step has time, omega, and predefined variables: alpha, J, theta, yaw, tilt
                        input_data = [
                            time[i], AOA[i], v_inf[i], b_data_array[i], ar_data_array[i], tr_data_array[i], sweep_data_array[i], 
                            gamma_data_array[i], cl_wing_calc[i], cd_wing_calc[i]
                        ]
                        # input_data = [
                        #     time[i], AOA[i], v_inf[i], b_data_array[i], ar_data_array[i], tr_data_array[i], sweep_data_array[i], 
                        #     gamma_data_array[i]#, cl_wing_calc[i], cd_wing_calc[i]
                        # ]
                        # output_data = [fft_cl_real[i], fft_cl_imag[i], fft_cd_real[i], fft_cd_imag[i]]
                        output_data = [cl[i], cd[i]]
                        
                        
                        sequence_inputs.append(input_data)
                        sequence_outputs.append(output_data)

                    sequence_inputs = np.array([sequence_inputs], dtype=float)
                    sequence_outputs = np.array([sequence_outputs], dtype=float)

                    # Append input sequence (n_timesteps, num_features) and output (Cl, Cd)
                    self.data.append(sequence_inputs)
                    self.targets.append(sequence_outputs)  # Append the whole Cl and Cd sequences


    def __len__(self):
        """
        Returns the total number of sequences in the dataset.
        """
        return len(self.data)

    def __getitem__(self, idx):
        """
        Returns a single sequence and its targets.
        """
        inputs = self.data[idx]  # Input sequence: (n_timesteps, n_features)
        targets = self.targets[idx]  # Output: (n_timesteps, 2)
        # return inputs, targets
        return torch.tensor(inputs, dtype=torch.float32), torch.tensor(targets, dtype=torch.float32)

    def get_variable(self, variable_name):
        """
        Returns a list of arrays for the specified variable.
        Args:
            'time' - timesteps
            
        """
        if variable_name == 'time':
            return self.time_data  # Return all time steps for each simulation
        elif variable_name == 'CL':
            return self.cl_data  # Return all omega (RPM) values for each simulation
        elif variable_name == 'CD':
            return self.cd_data
        elif variable_name == 'AOA':
            return self.AOA_data
        elif variable_name == 'Vinf':
            return self.v_inf_data  
        elif variable_name == 'fft_cl':
            return self.fft_cl_r, self.fft_cl_i 
        elif variable_name == 'fft_cd':
            return self.fft_cd_r, self.fft_cd_i 
        else:
            raise ValueError(f"Variable {variable_name} not supported.")

In [ ]:
class LSTMNetWithAttention(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, num_heads, bias=0.0):
        super(LSTMNetWithAttention, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bias = nn.Parameter(torch.tensor(bias))  # Learnable bias term

        # LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=0.2)

        # Multi-head attention layer
        self.multihead_attention = nn.MultiheadAttention(hidden_size, num_heads, batch_first=True)

        # Fully connected output layer
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        """
        x: Input tensor of shape (batch_size, seq_length, input_size)
        """
        # Pass through LSTM
        lstm_out, _ = self.lstm(x)  # lstm_out: (batch_size, seq_length, hidden_size)

        # Multi-head attention: Query, Key, and Value are all lstm_out
        attention_output, attention_weights = self.multihead_attention(lstm_out, lstm_out, lstm_out)

        # Pass attention output through the fully connected layer
        output = self.fc(attention_output)  # (batch_size, seq_length, output_size)

        # Add bias to the output
        output = output + self.bias

        return output, attention_weights

In [ ]:
from wing_static import LSTMNet_static

# Static Model
input_size_wing_stat = 10           # Number of input features
hidden_size_wing_stat = 50          # Hidden LSTM cells
output_size_wing_stat = 2           # Number of output features
num_layers_wing_stat = 2            # Number of LSTM layers
num_heads = 2                       # Number of attention heads

class WingModel_static(LSTMNetWithAttention):
    def __init__(self):
        super(WingModel_static, self).__init__(input_size_wing_stat, hidden_size_wing_stat, output_size_wing_stat, num_layers_wing_stat, num_heads)

# Initialize the model

root_wingModelsTrained = model_path + '/models/wing/'
root_wingScalersTrained = model_path + '/scalers/wing/'

# /mnt/e/eVTOL_model/eVTOL-VehicleModel/trained_models/models/wing/2025-03-07_LSTM_eMO_wingModel_static_with_coeff_lr0.0005_e2000_nL2_numNN50.pth
wing_model_static = WingModel_static()
wing_model_static.load_state_dict(torch.load(root_wingModelsTrained+'2025-03-07_LSTM_eMO_wingModel_static_with_coeff_lr0.0005_e2000_nL2_numNN50.pth'))
wing_model_static = wing_model_static.to(device)
wing_model_static.eval()

# Load the scaler
# /mnt/e/eVTOL_model/eVTOL-VehicleModel/trained_models/scalers/wing/2025-02-26_LSTM_eMO_wingModel_static_ipScaler_lr0.001_e2000_nL2_numNN50.pkl

# /mnt/e/eVTOL_model/eVTOL-VehicleModel/trained_models/scalers/wing/2025-03-07_LSTM_eMO_wingModel_static_with_coeff_ipScaler_lr0.0005_e2000_nL2_numNN50.pkl
input_scaler_wing_stat = joblib.load(root_wingScalersTrained+'2025-03-07_LSTM_eMO_wingModel_static_with_coeff_ipScaler_lr0.0005_e2000_nL2_numNN50.pkl')
output_scaler_wing_stat = joblib.load(root_wingScalersTrained+'2025-03-07_LSTM_eMO_wingModel_static_with_coeff_opScaler_lr0.0005_e2000_nL2_numNN50.pkl')

In [ ]:
from sklearn.metrics import r2_score

root_test_base = data_path + '/testing_data/'

def mape(y_true, y_pred):
    """Compute Mean Absolute Percentage Error (MAPE)"""
    mask = y_true != 0  # Avoid division by zero
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

def relative_l2_norm(y_true, y_pred):
    """Compute Relative L2 Norm Error (ε)"""
    mask = y_true != 0  # Avoid division by zero
    numerator = np.linalg.norm(y_pred[mask] - y_true[mask], ord=2)  # ||pred - true||_2
    denominator = np.linalg.norm(y_true[mask], ord=2)  # ||true||_2
    return (numerator / denominator) * 100

# Initialize lists to store evaluation results
mape_cl_list, mape_cd_list = [], []
r2_cl_list, r2_cd_list = [], []
relative_l2_norm_cl_list, relative_l2_norm_cd_list = [], []
case_names = []

for simulation_case in os.listdir(root_test_base):
    root_dir_test_sim = root_test_base + simulation_case

    # Load dataset
    dataset_test = WingDataset(root_dir_test_sim, subdir_condition=subdir_condition)
    inputs_test, outputs_test = dataset_test[0:]

    input_tensor_test = inputs_test.squeeze(1)  # Remove singleton dimension
    output_tensor_test = outputs_test.squeeze(1)

    # Normalize inputs
    inputs_test_reshaped = input_tensor_test.reshape(-1, input_size_wing_stat)
    test_inputs_normalized = input_scaler_wing_stat.transform(inputs_test_reshaped).reshape(input_tensor_test.shape)
    test_inputs_tensor = torch.tensor(test_inputs_normalized, dtype=torch.float32).to(device)

    # Model prediction
    wing_model_static.eval()
    with torch.no_grad():
        predicted_outputs, _ = wing_model_static(test_inputs_tensor)  # Assuming model returns attention weights too

    # Convert predictions back to original scale
    predicted_outputs = predicted_outputs.cpu().detach().numpy()
    predicted_outputs_original_scale = output_scaler_wing_stat.inverse_transform(predicted_outputs.reshape(-1, output_size_wing_stat))
    predicted_outputs_original_scale = predicted_outputs_original_scale.reshape(input_tensor_test.shape[0], input_tensor_test.shape[1], output_size_wing_stat)
    predicted_outputs_original_scale = predicted_outputs_original_scale[0]  # Select first batch if needed

    # Extract Cl and Cd predictions
    cl_test_NN = predicted_outputs_original_scale[:, 0]
    cd_test_NN = predicted_outputs_original_scale[:, 1]

    # Load ground truth (FLOWUnsteady simulation)
    time = dataset_test.get_variable('time')[0]
    cl_test_flowuns = dataset_test.get_variable('CL')[0]  # Ensure correct shape
    cd_test_flowuns = dataset_test.get_variable('CD')[0]

    # Compute MAPE and R² Score
    mape_cl = mape(cl_test_flowuns, cl_test_NN)
    mape_cd = mape(cd_test_flowuns, cd_test_NN)
    r2_cl = r2_score(cl_test_flowuns, cl_test_NN)
    r2_cd = r2_score(cd_test_flowuns, cd_test_NN)
    relative_l2_norm_cl = relative_l2_norm(cl_test_flowuns, cl_test_NN)
    relative_l2_norm_cd = relative_l2_norm(cd_test_flowuns, cd_test_NN)


    # Store results
    mape_cl_list.append(mape_cl)
    mape_cd_list.append(mape_cd)
    r2_cl_list.append(r2_cl)
    r2_cd_list.append(r2_cd)
    relative_l2_norm_cl_list.append(relative_l2_norm_cl)
    relative_l2_norm_cd_list.append(relative_l2_norm_cd)
    case_names.append(simulation_case)

    if PLOT_RESULT:
        plt.figure(figsize=(15, 5))
        legend_labels = simulation_case.split('_')
        aoa_label = legend_labels[4]
        v_inf_label = legend_labels[6]
        
        # Plot for Lift coefficient, $C_L$
        plt.subplot(1, 2, 1)
        plt.plot(time, cl_test_NN, label='Predicted $C_L$', color='black', linestyle='--', linewidth=2)
        plt.plot(time, cl_test_flowuns, label='Actual $C_L$', color='red', linestyle='-', linewidth=2)
        plt.xlabel('Time [s]', fontsize=16)
        plt.ylabel('Lift coefficient, $C_L$', fontsize=16)
        plt.title(f'Lift Coefficient Comparison', fontsize=20)
        plt.legend(loc='upper center', fontsize=16, title='$AoA$ = ${}^{{\circ}}$, $V_{{\infty}}$ = ${} m/s$'.format(aoa_label, v_inf_label), fancybox=True, borderpad=1, title_fontsize='16', ncol=2)
        plt.ylim([min(cl_test_NN.min(), cl_test_flowuns.min()) - 0.05, max(cl_test_NN.max(), cl_test_flowuns.max()) + 0.25])
        plt.grid(True, linestyle='--', linewidth=0.5)
        
        # Plot for Drag coefficient, $C_D$
        plt.subplot(1, 2, 2)
        plt.plot(time, cd_test_NN, label='Predicted $C_D$', color='black', linestyle='--', linewidth=2)
        plt.plot(time, cd_test_flowuns, label='Actual $C_D$', color='red', linestyle='-', linewidth=2)
        plt.xlabel('Time [s]', fontsize=16)
        plt.ylabel('Drag coefficient, $C_D$', fontsize=16)
        plt.title(f'Drag Coefficient Comparison', fontsize=20)
        plt.legend(loc='upper center', fontsize=16, title='$AoA$ = ${}^{{\circ}}$, $V_{{\infty}}$ = ${} m/s$'.format(aoa_label, v_inf_label), fancybox=True, borderpad=1, title_fontsize='16', ncol=2)
        plt.ylim([min(cd_test_NN.min(), cd_test_flowuns.min()) - 0.0002, max(cd_test_NN.max(), cd_test_flowuns.max()) + 0.007])
        plt.grid(True, linestyle='--', linewidth=0.5)
        
        plt.tight_layout()
        if SAVE_PLOT:
            plt.savefig(f"{save_path}{simulation_case}_comparison.pdf", format="pdf", dpi=300, bbox_inches="tight")
        plt.show()

# Compute average metrics across all test cases
print("\nOverall Metrics Across All Test Cases:")
print(f"  Avg MAPE Cl: {np.mean(mape_cl_list):.2f}%, Avg MAPE Cd: {np.mean(mape_cd_list):.2f}%")
print(f"  Avg R² Cl: {np.mean(r2_cl_list):.4f}, Avg R² Cd: {np.mean(r2_cd_list):.4f}")
print(f"  Avg ε Cl: {np.mean(relative_l2_norm_cl_list):.2f}%, Avg ε Cd: {np.mean(relative_l2_norm_cd_list):.2f}%")

if SAVE_RESULT:
    # Convert results to a DataFrame
    results_df = pd.DataFrame({
        "Case": case_names,
        "Relative L2 Norm Cl (%)": relative_l2_norm_cl_list,
        "Relative L2 Norm Cd (%)": relative_l2_norm_cd_list,
        "MAPE Cl (%)": mape_cl_list,
        "MAPE Cd (%)": mape_cd_list,
        "R² Cl": r2_cl_list,
        "R² Cd": r2_cd_list
    })

    # Append mean values to the DataFrame
    mean_row = pd.DataFrame({
        "Case": ["Mean"], 
        "Relative L2 Norm Cl (%)": [np.mean(relative_l2_norm_cl_list)],
        "Relative L2 Norm Cd (%)": [np.mean(relative_l2_norm_cd_list)],
        "MAPE Cl (%)": [np.mean(mape_cl_list)],
        "MAPE Cd (%)": [np.mean(mape_cd_list)],
        "R² Cl": [np.mean(r2_cl_list)],
        "R² Cd": [np.mean(r2_cd_list)]
    })

    results_df = pd.concat([results_df, mean_row], ignore_index=True)

    # Define CSV file path
    csv_filename = os.path.join(save_path, "wingModel_wo_coeff_evaluation_results.csv")

    # Save to CSV
    results_df.to_csv(csv_filename, index=False)

    print(f"Results (including mean values) saved to {csv_filename}")